In [15]:
import os,sys

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score, recall_score, roc_auc_score, confusion_matrix
from collections import Counter
import scipy.io



In [19]:
from loc_utils import VolumeVisualizer

In [15]:
'''The features for this demo are stored as part of a .mat file that places patient and feature as a dataframe. along with the labels.
For this example the training and testing splits have alread been defined   

Inflammation
training=[1	8	9	12	16	22	26	34	35	37	39	42	47	49	51	2	7	13	14	18	20	21	23	30	32	33	36	38	40	41	43	44	45	50]
testing=[31	11	46	29	5	10	27	19	48	15	3	24	25	28	4	6	17]
"H:\My Drive\Research\MREcrohnsproject\Data\Florian_processed\3DFeats\Stack\Strict_Crohns_TI_featstack_multiROI-v3_wc.mat"
                featstack_TI=featstack_mre_SRmed(training,(1:end-3));
                labels=featstack_mre_SRmed(training,(end-2:end));

Fibrosis

 training=[3,16,17,20,24,27,31,34,36,40,41,46,49,51,1,4,6,7,8,10,11,13,14,15,18,21,22,26,29,30,32,33,38,45]
 testing=[39,50,43,9,35,44,12,25,23,48,42,37,5,19,47,28,2]
"H:\My Drive\Research\MREcrohnsproject\Data\Florian_processed\3DFeats\Stack\Strict_Crohns_TI_featstack_multiROI-v3_wc.mat"
                 featstack_TI=featstack_mre_SRmed(training,(1:end-3));
                labels=featstack_mre_SRmed(training,(end-2:end));
'''


ModuleNotFoundError: No module named 'loc_utils'

In [10]:
mat_data=scipy.io.loadmat(r'H:\My Drive\Research\MREcrohnsproject\Data\Florian_processed\3DFeats\Stack\Strict_Crohns_TI_featstack_multiROI-v3_wc.mat')



ImportError: attempted relative import with no known parent package